# Equation of state (EOS)
For the source code, see [eos](https://workgraph-collections.readthedocs.io/en/latest/qe/module.html#workgraph_collections.ase.espresso.eos.eos_workgraph).

## Visualizing the WorkGraph Builder




In [1]:
from workgraph_collections.qe.eos import eos_workgraph
from aiida import load_profile
load_profile()

task = eos_workgraph.TaskCls()
task.to_html()


## Visualizing the WorkGraph

In [2]:
from workgraph_collections.qe.eos import eos_workgraph
from aiida import load_profile
load_profile()

wg = eos_workgraph()
wg.to_html()

## Example: Silicon
### Prepare the inputs and submit the workflow


In [3]:
from aiida import load_profile, orm
from ase.build import bulk
from workgraph_collections.qe.eos import eos_workgraph

load_profile()

structure = orm.StructureData(ase=bulk("Si"))
code = orm.load_code("qe-7.2-pw@localhost")
parameters = {
    "CONTROL": {
        "calculation": "scf",
    },
    "SYSTEM": {
        "ecutwfc": 30,
        "ecutrho": 240,
        "occupations": "smearing",
        "smearing": "gaussian",
        "degauss": 0.1,
    },
}
# Load the pseudopotential family.
pseudo_family = orm.load_group("SSSP/1.3/PBEsol/efficiency")
pseudos = pseudo_family.get_pseudos(structure=structure)
kpoints = orm.KpointsData()
kpoints.set_kpoints_mesh([3, 3, 3])
#
metadata = {
    "options": {
        "resources": {
            "num_machines": 1,
            "num_mpiprocs_per_machine": 1,
        },
    }
}

# ===============================================================================
wg = eos_workgraph(
    structure=structure,
    code=code,
    scales=[0.98, 0.99, 1.0, 1.01, 1.02],
    parameters=parameters,
    kpoints=kpoints,
    pseudos=pseudos,
    metadata=metadata,
)
wg.name = "QE-EOS-Si"
#------------------------- Submit the calculation -------------------
wg.submit(wait=True, timeout=300)


WorkGraph process created, PK: 54748


<WorkChainNode: uuid: 7c17abf8-237d-4f47-98ad-3b54b77e5f36 (pk: 54748) (aiida_workgraph.engine.workgraph.WorkGraphEngine)>

## Print the results

In [7]:
#------------------------- Print the output -------------------------
data = wg.tasks["fit_eos"].outputs["result"].value.get_dict()
print('\nResult: \nB: {B}\nv0: {v0}\ne0: {e0}\nv0: {v0}'.format(**data))



Result: 
B: 83.930472220007
v0: 40.947349586558
e0: -308.18973212512
v0: 40.947349586558
